<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.955 · Models avançats de mineria de dades</p>
<p style="margin: 0; text-align:right;">2022-2 · Màster universitari en Ciència de dades (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# PAC 3: Mètodes supervisats

En aquesta pràctica veurem diferents mètodes supervisats i intentarem optimitzar diferents mètriques. Veurem com els diferents models classifiquen els punts i amb els quals obtenim més precisió. Després aplicarem tot allò que hem après fins ara a un dataset nou, simulant un cas pràctic real.

1. Exploració d'algorismes supervisats <br>
     1.0. Càrrega de dades <br>
     1.1. Naive-Bayes<br>
     1.2. Anàlisi Discriminant Lineal (LDA) i Anàlisi Discriminant Quadràtica (QDA)<br>
     1.3. K veïns més propers (KNN)<br>
     1.4. Màquines de suport vectorial (SVM)<br>
     1.5. Arbres de decisió 
2. Implementació del cas pràctic <br>
     2.0. Càrrega de dades<br>
     2.1. Anàlisi descriptiva<br>
     2.2. Entrenament, validació i prova d'una xarxa neuronal amb les dades originals<br>
     2.3. Submostreig<br>
     2.4. Sobremostreig<br>
     2.5. Generació de dades sintètiques<br>
     2.6. _Tuning_ dels models (BONUS)

<u>Consideracions generals</u>:

- La solució plantejada no pot utilitzar mètodes, funcions o paràmetres declarats **_deprecated_** en futures versions, a excepció de la càrrega de dades com s'indica posteriorment.
- Aquesta PAC s'ha de fer de manera **estrictament individual**. Qualsevol indici de còpia serà penalitzat amb un suspens (D) per a totes les parts implicades i la possible avaluació negativa de l'assignatura de forma íntegra.
- Cal que l'estudiant indiqui **totes les fonts** que ha fet servir per a la realització de la PAC. Si no és així, es considerarà que l'estudiant ha comès plagi, i és penalitzat amb un suspens (D) i la possible avaluació negativa de l'assignatura de forma íntegra.

<u>Format del lliurament</u>:

- Alguns exercicis poden suposar diversos minuts d'execució, per la qual cosa el lliurament s'ha de fer en **format notebook** i en **format html**, on es vegi el codi, els resultats i comentaris de cada exercici. Es pot exportar el notebook a HTML des del menú File $\to$ Download as $\to$ HTML.
- Hi ha un tipus de cel·la especial per contenir text. Aquest tipus de cel·la us serà molt útil per respondre a les diferents preguntes teòriques plantejades al llarg de l'activitat. Per canviar el tipus de cel·la a aquest tipus, aneu al menú: Cell $\to$ Cell Type $\to$ Markdown.

<div class="alert alert-block alert-info">
    <strong>Nom i cognoms:</strong>
</div>

In [ ]:
import numpy as np 
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


<a id='ex1'></a>
# 1. Exploració d'algorismes supervisats (5 punts)

## 1.0. Càrrega de dades

El conjunt de dades Spiral és un conjunt de dades sintètiques que s'utilitza habitualment en l'aprenentatge automàtic i la mineria de dades com un problema de classificació no lineal. El conjunt de dades consta de dues espirals entrellaçades que s'assemblen a les espirals d'Arquimedes. Cada espiral es compon d'un conjunt de punts distribuïts uniformement al pla, i els punts de cada espiral estan etiquetats amb una classe diferent.

L'objectiu és utilitzar un algorisme de classificació per predir la classe d'un punt desconegut en funció de les coordenades (x, y). A causa de la naturalesa entrellaçada de les espirals, aquest problema de classificació és no lineal i, per tant, és un desafiament interessant per als algorismes d'aprenentatge automàtic.

El codi següent carregarà 2000 punts a la variable `X` i la corresponent etiqueta o grup (en forma numèrica) a la variable `y`. Podem comprovar que la càrrega ha estat correcta obtenint les dimensions d'aquestes dues variables i el gràfic dels punts (amb colors diferents per a cada grup).

In [ ]:
data = pd.read_pickle('spiral.pickle')

# Convertir les dues primeres columnes en un array de numpy
X = data[['X1', 'X2']].values

# Convertir l'última columna en un array de numpy
y = data['y'].values

print('Dimensions de X', X.shape)
print('Dimensions de y', y.shape)

# Fer la representació gràfica
plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.viridis, alpha=0.5)
plt.show()

Al llarg dels exercicis aprendrem a veure gràficament les fronteres de decisió que ens tornen els diferents models. Per això utilitzarem la funció definida a continuació, que segueix els passos següents:

   - Crear una [meshgrid](https://docs.scipy.org/doc/numpy/reference/generated/numpy.meshgrid.html) amb els valors mínim i màxim de 'x' i 'y'.
   - Predir el classificador amb els valors de la _meshgrid_.
   - Fer un _reshape_ de les dades per tenir el format corresponent.
  
Un cop fet això, ja podem fer el gràfic de les fronteres de decisió i afegir-hi els punts reals. Així veurem les àrees que considera que el model són d'una classe i les que considera que són d'una altra. En sobreposar els punts, veurem si els classifica correctament a l'àrea que els correspon.

In [ ]:
def plot_decision_boundary(clf, X, Y, cmap='Paired'):
    if not isinstance(X, np.ndarray):  # Si X no és un array de numpy, el converteix
        X = X.to_numpy()
    
    h = 0.02
    x_min, x_max = X[:,0].min() - 10*h, X[:,0].max() + 10*h
    y_min, y_max = X[:,1].min() - 10*h, X[:,1].max() + 10*h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(7,7))
    plt.contourf(xx, yy, Z, cmap=cmap, alpha=0.25)
    plt.contour(xx, yy, Z, colors='k', linewidths=0.7)
    plt.scatter(X[:,0], X[:,1], c=Y, cmap=cmap, edgecolors='k', label=Y);

<div class="alert alert-block alert-info">
   <strong>Implementació:</strong>

Dividiu el _dataset_ en dos subconjunts, __*train*__ (80% de les dades) i __*test*__ (20% de les dades). Anomeneu els conjunts com: X_train, X_test, y_train, y_test. Utilitzeu l'opció `random_state = 24`.
    
Podeu utilitzar la implementació `train_test_split` de `sklearn`.
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

### 1.1. Gaussian Naïve Bayes

L'objectiu d'aquest primer exercici és entendre el funcionament de l'algorisme Naïve-Bayes, un algoritme peculiar, ja que es basa en el teorema de Bayes per calcular la probabilitat que una observació pertanyi a cadascuna de les classes. El model assumeix que les característiques d'entrada són independents entre si, cosa que permet simplificar el càlcul de les probabilitats condicionals.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un model de Naïve-Bayes. Podeu utilitzar el classificador `GaussianNB` de `sklearn`.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.
    
Podeu utilitzar la funció `plot_decision_boundary` creada prèviament, i les funcions `accuracy_score` i `confusion_matrix` del paquet `metrics` de `sklearn`.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algoritme utilitzat?
- Com són les prediccions obtingudes sobre el conjunt de test?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

### 1.2 Anàlisi Discriminant Lineal (LDA) i Anàlisi Discriminant Quadràtica (QDA)

Ara analitzarem dos algorismes que es basen en la transformació lineal de les característiques d'entrada per maximitzar la separació entre les classes. Aquests models suposen que les característiques tenen una distribució gaussiana, i així, poder calcular les probabilitats condicionals de cada classe i assignar la classe amb la major probabilitat com la classe predita per a una observació donada.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un model d'Anàlisi Discriminate Lineal (LDA). Podeu utilitzar el classificador `LinearDiscriminantAnalysis` de `sklearn`.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algoritme utilitzat?
- Com són les prediccions obtingudes sobre el conjunt de test?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong> 

- Amb el dataset de _train_, entreneu un model d'Anàlisi Discriminant Quadràtica (QDA). Podeu utilitzar el classificador `QuadraticDiscriminantAnalysis` de `sklearn`.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algoritme utilitzat?
- Com són les prediccions obtingudes sobre el conjunt de test?
- En què es diferencien l'algorisme LDA i QDA?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

### 1.3. K veïns més propers (KNN)

En aquest punt entendrem el funcionament de l'algorisme KNN (que es basa en la proximitat dels punts de dades en un espai de característiques), intuir-ne els principals avantatges o desavantatges i entendre la influència dels paràmetres de què està compost.

_K-Nearest Neighbor_ és un algorisme basat en instància de tipus supervisat.

Veurem què significa això:

   - Supervisat: tenim etiquetatge el nostre conjunt de dades d'entrenament, amb la classe o resultat esperat.
   - Basat en instància (_Lazy Learning_): Això significa que el nostre algorisme no aprèn explícitament un model (com per exemple en Regressió Logística o arbres de decisió), sinó que memoritza les instàncies d'entrenament que són utilitzades com a "coneixement" per a la fase de predicció.

Com funciona el KNN?

   - Calculem la distància entre l'ítem a classificar i els altres ítems del dataset d'entrenament.
   - Seleccionem els "k" elements més propers, és a dir, amb menor distància, segons la distància que definim (euclidiana, cosinus, manhattan, etc).
   - Finalment, fem una "votació de majoria" entre els k punts: els de la classe que "dominen" decidiran la seva classificació final.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un classificador KNN amb hiperparàmetre `n_neighbors=2`. Podeu utilitzar el classificador `KNeighborsClassifier` de `sklearn`.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.


Si en entrenar el classificador us apareix un _warning_ i el voleu ignorar, executeu el codi següent abans de l'entrenament:

> import warnings<br>
> warnings.filterwarnings('ignore', message='^.*will change.*$', category=FutureWarning)

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='^.*will change.*$', category=FutureWarning)

Al model entrenat, hem fixat el paràmetre `n_neighbors` de forma arbitrària. Però podria ser que amb un altre valor obtinguéssim una millor predicció.

Per conèixer el valor òptim dels paràmetres d'un model (_hyperparameter tunning_) se sol utilitzar una cerca de reixeta (_grid search_). És a dir, entrenar un model per a cada combinació d'hiperparàmetres possible i avaluar-lo fent servir validació creuada (_cross validation_) amb 5 particions estratificades. Posteriorment, s'escull la combinació d'hiperparàmetres que hagi obtingut millors resultats.

En aquest cas, només volem optimitzar un hiperparàmetre:

- _k_: el nombre de veïns que es consideren per classificar un nou exemple. Provarem amb tots els valors entre 1 i 20.

<div class="alert alert-block alert-info">
    <strong>Implementació:</strong>

Calculeu del valor òptim de l'hiperparàmetre _k_ (`n_neighbors`). Utilitzeu una cerca de reixeta amb validació creuada per trobar el valor òptim de _k_. Per cada valor, calculeu la seva mitjana i la desviació estàndard. Implementeu un _heatmap_ per visualitzar la precisió segons els diferents valors de l'hiperparàmetre.
    
Podeu utilitzar el mòdul `GridSearchCV` de `sklearn` pel càlcul del millor hiperparàmetre, i `heatmap` de `Seaborn`.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un classificador KNN amb el millor hiperparàmetre trobat.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Comentar els resultats de la cerca del millor hiperparàmetre.
- Com es visualitza gràficament el canvi del valor `n_neighbors`? Té sentit aquesta diferència entre els dos gràfics en canviar el paràmetre?
- Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algoritme utilitzat?
- Com són les prediccions obtingudes sobre el conjunt de test?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

### 1.4. Màquines de suport vectorial (SVM)

Les _Support Vector Machine (SVM)_ es fonamenten en el _Maximal Margin Classifier_, que alhora es basa en el concepte d'hiperplà.

En un espai p-dimensional, un hiperplà es defineix com un subespai pla i afí de dimensions p-1. El terme afí significa que el subespai no ha de passar per l'origen. En un espai de dues dimensions, l'hiperplà és un subespai d'una dimensió, és a dir, una recta. En un espai tridimensional, un hiperplà és un subespai de dues dimensions, és a dir, un pla convencional. Per a dimensions p>3 no és intuïtiu visualitzar un hiperplà, però el concepte de subespai amb p-1 dimensions es manté.

La definició d'hiperplà per a casos perfectament separables linealment resulta en un nombre infinit de possibles hiperplans, cosa que fa necessari un mètode que permeti seleccionar-ne un com a classificador òptim.

La solució a aquest problema consisteix a seleccionar com a classificador òptim el que es coneix com a _maximal margin hyperplane_ o hiperplà òptim de separació, que es correspon amb l'hiperplà que es troba més allunyat de totes les observacions d'entrenament. Per obtenir-lo, cal calcular la distància perpendicular de cada observació a un determinat hiperplà. La menor d'aquestes distàncies (coneguda com a marge) determina com de lluny hi ha l'hiperplà de les observacions d'entrenament. El _maximal margin hyperplane_ es defineix com l'hiperplà que aconsegueix un marge més gran, és a dir, que la distància mínima entre l'hiperplà i les observacions és el més gran possible. Encara que aquesta idea sona raonable, no és possible aplicar-la, ja que hi hauria infinits hiperplans contra els quals mesurar les distàncies. Per resoldre-ho, es recórrer a mètodes d'optimització.

El procés d'optimització té la peculiaritat que només les observacions que es troben just al marge o que el violen influeixen sobre l'hiperplà. A aquestes observacions se'ls coneix com a vectors suport (_vectors suport_) i són les que defineixen el classificador obtingut.

#### Els _kernels_ en SVM

En alguns casos, no hi ha manera de trobar un hiperplà que permeti separar dues classes. En aquests casos diem que les classes no són linealment separables. Per resoldre aquest problema podem utilitzar el truc del _kernel_.

El truc del _kernel_ (_kernel trick_) consisteix a utilitzar una dimensió nova en què puguem trobar un hiperplà per separar les classes. Es pot veure un exemple a: https://www.youtube.com/watch?v=OdlNM96sHio

Igual que en l'algorisme vist anteriorment (KNN), les SVM també depenen de diversos hiperparàmetres.

En aquest cas intentarem optimitzar dos hiperparàmetres:

- **C**: és la regularització, és a dir, el valor de penalització dels errors a la classificació. Indica el compromís entre obtenir l'hiperplà amb el marge més gran possible i classificar-ne el màxim nombre d'exemples correctament. Provarem els valors: 0.01, 0.1, 1, 10, 50, 100 i 200.
  
- **Gama**: coeficient que multiplica la distància entre dos punts al nucli radial. Per dir-ho a "grosso modo", com més petit és gamma, més influència tenen dos punts propers. Provarem els valors: 0.001, 0.01, 0.1, 1 i 10.
  
Igual que en el cas anterior, per validar el rendiment de l'algorisme amb cada combinació d'hiperparàmetres farem servir validació creuada (_cross-validation_) amb 4 particions estratificades.

<div class="alert alert-block alert-info">
    <strong>Implementació:</strong>

Calculeu del valor òptim dels hiperparàmetres _C_ i _gama_. Utilitzeu una cerca de reixeta amb validació creuada per trobar els valors òptims. Per a cada combinació de valors, calculeu la seva mitjana i la desviació estàndard. Feu un _heatmap_ per visualitzar la precisió segons els diferents valors dels hiperparàmetres.

Podeu utilitzar el mòdul `GridSearchCV` de `sklearn` pel càlcul dels millors hiperparàmetres amb el classificador SVC (de `SVM` de `sklearn`), i `heatmap` de `Seaborn`.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un model de SVM amb la millor combinació de paràmetres trobada.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Comentar els resultats de la cerca dels millors hiperparàmetres.
- Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algoritme utilitzat?
- Com són les prediccions obtingudes sobre el conjunt de test?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

### 1.5. Arbres de decisió

Els arbres de decisió són models predictius formats per regles binàries (si/no) amb què s'aconsegueix repartir les observacions en funció dels seus atributs i predir el valor de la variable resposta.

Els arbres poden ser **classificadors** (per classificar classes, tals com el nostre exemple), o bé **regressors** (per predir variables contínues).

#### Construcció d´un arbre

La creació de les ramificacions dels arbres s'aconsegueix mitjançant l'algorisme de *recursive binary splitting*. Aquest algorisme consta de tres passos principals:

- El procés s'inicia a dalt de tot de l'arbre, on totes les observacions pertanyen a la mateixa regió.
- S'identifiquen tots els possibles punts de tall per a cadascun dels predictors. Els punts de tall són cadascun dels nivells.
- S'avaluen les possibles divisions de cada predictor d'acord amb una mesura determinada. En el cas dels classificadors s'utilitzen: *classification error rate*, Gini, entropia, txi-square.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un arbre de decisió. Podeu utilitzar el classificador `DecisionTreeClassifier` (de `tree` de `sklearn`).

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.
    
- Representeu l'arbre. Podeu utilitzar la comanda `plot.tree` de la biblioteca `tree` de `sklearn`.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Comenteu els resultats.

</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

#### Evitant el *overfitting*

El procés de construcció d'arbres descrit tendeix a reduir ràpidament l'error d'entrenament, per la qual cosa generalment el model s'ajusta molt bé a les observacions utilitzades com a entrenament (conjunt de train). Com a conseqüència, els arbres de decisió tendeixen al *overfitting*.

Per prevenir-ho, utilitzarem dos hiperparàmetres:

- `max_depth`: la profunditat màxima de l'arbre. Explorarem els valors entre 4 i 10.
- `min_samples_split`: el nombre mínim d'observacions que ha de tenir un full de l'arbre per poder dividir-lo. Explorarem els valors: 2, 10, 20, 50 i 100.

<div class="alert alert-block alert-info">
    <strong>Implementación:</strong>

Calculeu el valor òptim dels hiperparàmetres `max_depth` i `min_samples_split`. Utilitzeu una cerca de reixeta amb validació creuada per trobar els valors òptims. Per a cada combinació de valors, calculeu la seva mitjana i la desviació estàndard. Feu un _heatmap_ per visualitzar la precisió segons els diferents valors dels hiperparàmetres.
    
Podeu utilitzar el mòdul `GridSearchCV` de `sklearn` pel càlcul dels millors hiperparàmetres amb el classificador `DecisionTreeClassifier` (de `tree` de `sklearn`), i `heatmap` de `Seaborn`.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Amb el dataset de _train_, entreneu un arbre de decisió amb la millor combinació de paràmetres trobada.

- Calculeu el _accuracy_ del model obtingut sobre _train_ i _test_.

- Calculeu la matriu de confusió sobre _test_.
    
- Representeu gràficament la frontera de decisió amb el conjunt de _test_.
    
- Representeu l'arbre.
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Comentar els resultats de la cerca dels millors hiperparàmetres.
- Com són les fronteres de decisió? Té sentit que tinguin aquesta forma amb l'algoritme utilitzat?
- Com són les prediccions obtingudes sobre el conjunt de test?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

<a id='ej2'></a>
# 2. Implementació del cas pràctic (5 punts)

Com a expert en anàlisi de dades, sabem la importància que les empreses de targetes de crèdit puguin identificar i prevenir transaccions fraudulentes per protegir els seus clients. En aquest sentit, estudiarem un conjunt de dades que conté informació sobre transaccions realitzades amb targetes de crèdit el setembre del 2013 per titulars de targetes europeus.

Aquest conjunt de dades presenta transaccions efectuades en dos dies, on s'han registrat 492 casos de frau d'un total de 284.807 transaccions. És important destacar que totes les variables d'entrada són numèriques i van ser obtingudes a través duna transformació PCA. Lamentablement, per raons de confidencialitat, no es poden proporcionar les característiques originals, ni més informació sobre les dades. 

Les variables V1 a V28 representen els components principals obtinguts amb PCA, mentre que “Time” i “Amount” són les úniques variables que no han estat transformades amb PCA. La variable "Time" indica els segons transcorreguts entre cada transacció i la primera transacció del conjunt de dades, mentre que "Amount" representa la suma de la transacció. La variable “Class” és la variable de resposta i pren el valor 1 en cas de frau i 0 en cas contrari.

Font: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

## 2.0. Càrrega de dades

El primer que farem serà carregar el conjunt de dades, visualitzar les primeres files i verificar:

- La quantitat total de files i columnes al DataFrame.
- El nom de cada columna del DataFrame.
- El nombre de valors no nuls a cada columna.
- El tipus de dades de cada columna, que poden ser int, float, object, entre d'altres.
- La quantitat de memòria utilitzada pel DataFrame.

In [ ]:
import numpy as np 
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
data.head()

In [ ]:
data.info()

## 2.1. Preprocessament de dades i anàlisi exploratòria

El preprocessament i l'anàlisi exploratòria de les dades és un pas crític i fonamental en qualsevol projecte d'anàlisi de dades o d'aprenentatge automàtic. Ajuda als investigadors a comprendre millor les dades, descobrir patrons i relacions, identificar problemes i seleccionar les tècniques d'anàlisi adequades per al conjunt de dades

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Calculeu les freqüències de la variable _target_ (`Class`) i feu un gràfic de barres.

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

Analitzar la distribució de les variables descriptores. Representa gràficament l'histograma de les 30 variables separant les observacions segons la classe a què pertany. Organitza tots els histogrames a 10 files i 3 columnes.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Com és la relació de les freqüències de la variable `Class`?
- Quina informació ens proporcionen els histogrames? Hi ha una altra forma de visualització que pogués ser útil, en aquest cas?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Les dades en brut presenten alguns problemes. En primer lloc, les columnes `Time` i `Amount` són massa variables per utilitzar-les directament. Elimineu la columna `Time` (ja que no és clar què significa) i preneu el logaritme de la columna `Amount` per reduir el seu rang.
    
Per evitar la indeterminació "log(0)", sumeu-li 1 cèntim de dolar (0.001) a la columna `Amount` abans de calcular el logaritme. No oblideu que finalment heu de reemplaçar la columna `Amount` per `Log Amount`
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Separeu els descriptors de la resposta. Anomeneu els conjunts com: X, y.

- Dividiu el _dataset_ en dos subconjunts, __*train*__ (80% de les dades) i __*test*__ (20% de les dades). Anomeneu els conjunts com: X_train, X_test, y_train, y_test. Podeu utilitzar la funció `train_test_split` de la biblioteca `model_selection` de `sklearn`. Utilitzeu l'opció `random_state = 24` i assegureu-vos que la divisió sigui estratificada, és a dir, que es mantingui la mateixa proporció de classes tant en el conjunt d'entrenament com en el conjunt de test.

Tingueu en compte que les matrius de les classes `y_train` i `y_test` han d'estar codificades. La funció `to_categorical` de la llibreria `TensorFlow` de `Keras` s'utilitza per convertir una matriu d'etiquetes de classe (sencers) en una matriu d'etiquetes de classe codificades en _one-hot_.

La codificació _one-hot_ és un procés mitjançant el qual les etiquetes categòriques es converteixen en vectors binaris on cada vector té una longitud igual al nombre de classes. Cada vector té un valor d'1 a la posició corresponent a la classe i un valor de 0 a totes les altres posicions. Això es fa per permetre que els models d'aprenentatge automàtic comprenguin millor l'estructura de les etiquetes categòriques.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Normalitzeu els descriptors utilitzant `StandardScaler` de `sklearn`.
- Mostreu les dimensions del conjunt de descriptors original, del conjunt d'entrenament i del conjunt de test.
    
<strong>Nota:</strong> Recordeu que el `StandardScaler` només s'ajusta utilitzant els descriptors d'entrenament per evitar fuites d'informació o "data leakage". La fuita d'informació es produeix quan es fa servir informació de les dades de test o validació per ajustar el model. En altres paraules, si s'ajusta el model d'escalat amb tot el conjunt de dades, s'utilitzaria informació de test o validació per a l'ajust, cosa que podria fer que el model sembli més precís del que realment és.

</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

## 2.2. Entrenament, validació i prova d'una xarxa neuronal amb les dades originals

Com a eina de classificació que ens permeti predir si una transacció és fraudulenta o no, farem servir un perceptró multicapa. 

Un perceptró multicapa (MLP, per les sigles en anglès) és una xarxa neuronal artificial composta per múltiples capes d'unitats de processament (neurones), on cada capa està connectada a la següent capa a través d'un conjunt de connexions ponderades. El MLP és capaç de realitzar tant tasques de classificació com de regressió, en aprendre a mapejar les entrades a les sortides desitjades mitjançant una funció d'activació no lineal. 

La xarxa utilitza un algoritme d'aprenentatge supervisat que ajusta els pesos de les connexions durant l'entrenament per minimitzar la diferència entre les sortides produïdes per la xarxa i les sortides desitjades. A causa de la seva capacitat per modelar relacions no lineals complexes, el MLP és un dels models més utilitzats al camp de l'aprenentatge automàtic.

Crearem i entrenarem una xarxa Perceptró Multicapa (MLP) amb 4 capes ocultes de 20 neurones cadascuna i amb funció dʻactivació `relu`. Farem servir la classe `Sequential` de la llibreria `keras` per crear el model de forma seqüencial, és a dir, apilant capes una damunt de l'altra. Sequential és la forma més senzilla de crear models de xarxes neuronals en keras, ja que no requereix definir la direcció del graf computacional com passa en altres tipus de models més complexos.


<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Creeu el model `Sequential`. Afegiu les 4 capes ocultes amb 20 neurones cadascuna utilitzant el mètode `.add()` i funció d'activació `relu`. Afegiu la capa de sortida amb 2 neurones de sortida (una per a la classe 0 i l'altra per a la classe 1) i funció d'activació `sigmoid`. Mostreu el resum de la xarxa creada amb el mètode `.summary()`
- Compileu el model utilitzant el mètode `.compile()`, especificant l'optimitzador `adam`, la funció de pèrdua `binary_crossentropy` i les mètrica d'avaluació `accuracy`.
- Entreneu el model utilitzant el mètode `.fit()` amb `X_train`, especifiqueu el nombre d'èpoques en 100 i la mida del lot el 2048. Valideu el vostre rendiment fent validació creuada amb el 80% de les dades per a l'entrenament. Configura el paràmetre `validation_split=0.2`.
- Grafiqueu la pèrdua (`loss`) tant d'entrenament com de validació en funció de les èpoques.
- Grafiqueu l'exactitud (`accuracy`) tant d'entrenament com de validació en funció de les èpoques.
    
<strong>Nota:</strong> Per a la presentació de l'informe, configureu el paràmetre `verbose=0`, per evitar mostrar informació durant l'entrenament.
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

Un cop entrenat i validat el model, procedirem a fer la prova, és a dir, predir la classe de `X_test` i calcular les mesures de rendiment.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Prediu la classe de `X_test`, calculeu l'exacitud de la predicció.
- Calculeu la matriu de confusió
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Què opines dels resultats?, ens podem quedar amb aquest model com a acceptable?
- **Les mesures de rendiment usades són acceptables?** Quina altra mesura de rendiment proposaries?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

## 2.3. Submostreig

El submostreig fa referència a la tècnica de reduir el nombre de mostres de la classe majoritària per equilibrar la distribució de classes en un conjunt de dades. Això es pot aconseguir eliminant aleatòriament mostres de la classe majoritària.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Redueix el nombre d'observacions de la classe majoritària del conjunt de dades _X_train_ fins que tingui el mateix nombre d'observacions que la classe minoritària. Mostra les dimensions de les noves matrius d'entrenament i la freqüència de cada classe. Podeu utilitzar la fució `resample` de la bibioteca `utils` de `sklearn`.
- Creeu, entreneu, grafiqueu les pèrdues i exactitud durant l'entrenament, valitadeu i proveu un altre model Sequential amb la mateixa configuració de l'apartat anterior.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Per què creus que s'ha fet un remostreig només a les dades d'entrenament?
- Què opines dels resultats? Ens podem quedar amb aquest model com a acceptable?
    
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

## 2.4. Sobremostreig

Al contrari que el cas anterior, el sobremostreig fa referència a la tècnica d'augmentar el nombre de mostres de la classe minoritària per equilibrar la distribució de classes en un conjunt de dades. Això es pot aconseguir mitjançant la replicació de mostres existents o mitjançant la generació de mostres sintètiques de les classes minoritàries.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Augmenta el nombre d'observacions de la classe minoritària del conjunt de dades _X_train_ fins que tingui el mateix nombre d'observacions que la classe majoritària. Mostra les dimensions de les noves matrius d'entrenament i la freqüència de cada classe.
- Creeu, entreneu i grafiqueu les pèrdues i exactitud durant l'entrenament, valitadeu i proveu un altre model Sequential amb la mateixa configuració de l'apartat anterior.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Per què creus que s'ha fet un remostreig només a les dades d'entrenament?
- Què opines dels resultats? Ens podem quedar amb aquest model com a acceptable?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

## 2.5. Generació de dades sintètiques

SMOTE (Synthetic Minority Over-sampling Technique) és una tècnica de sobremostreig utilitzada en l'aprenentatge automàtic per abordar el problema de classes desequilibrades. SMOTE es fa servir per augmentar el nombre de mostres de la classe minoritària en generar noves mostres sintètiques.

La tècnica SMOTE funciona de la manera següent: 
- Per a cada mostra a la classe minoritària, SMOTE selecciona k veïns propers i crea noves mostres a l'espai entre la mostra i els seus veïns. 
- Aquestes noves mostres sintètiques són agregades al conjunt de dades per augmentar el nombre de mostres de la classe minoritària.

La tècnica SMOTE s'utilitza en combinació amb altres tècniques de preprocessament, com ara el submostreig i la validació creuada estratificada, per abordar el problema de classes desequilibrades en problemes de classificació. Aquesta tècnica pot millorar la capacitat del model per aprendre patrons de les classes minoritàries i pot augmentar el rendiment a la classificació de classes minoritàries.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Mitjançant la tècnica SMOTE, augmenta el nombre d'observacions de la classe minoritària de tot el conjunt de dades `X`. Pots utilitzar la funció `SMOTE` de la llibreria `over_sampling` de la bibioteca `imblearn`. Mostra les dimensions de les matrius d'entrenament noves i la freqüència de cada classe.
- Creeu, entreneu i grafiqueu les pèrdues i exactitud durant l'entrenament, valitadeu i proveu un altre model Sequential amb la mateixa configuració de l'apartat anterior.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Anàlisi:</strong>

- Per què creus que en aquest cas si es pot fer el remostreig a tot el conjunt de dades?
- Què opines dels resultats? Aquest és millor que l'anterior model? Ens podem quedar amb aquest model com a acceptable?
- Enfocant-nos a les variables descriptores, com creus que es pot millorar la predicció?
</div>

<div class="alert alert-block alert-success">
<strong>Resposta:</strong> 


</div>

## 2.6. _Tuning_ dels models (BONUS)

Els models anteriors s'han configurat amb hiperparàmetres escollits arbitràriament, i potser no generen el model amb el millor rendiment. La recerca d'hiperparàmetres òptims és important perquè permet trobar la combinació de paràmetres que maximitza el rendiment del model en les dades de prova o validació, cosa que proporciona un model més generalitzable i precís per a noves dades.

<div class="alert alert-block alert-info">
<strong>Implementació:</strong>

- Mitjançant la funció `GridSearchCV` de `sklearn.model_selection` i la funció `KerasClassifier` de `keras.wrappers.scikit_learn`, troba els hiperparàmetres òptims per als models amb els conjunt de dades: submostrejat, sobremostrejat i amb generació sintètica per mitjà de SMOTE.
</div>

<div class="alert alert-block alert-danger">
<strong>Solució:</strong>
</div>